In [43]:
import os
import sys

# Ensure the current directory is in the Python path
sys.path.append('/Users/iyongjeong/WORK/dacon/clf/')
from loader import ImageDataset
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet50, ResNet50_Weights, resnet18, ResNet18_Weights, efficientnet_b0, EfficientNet_B0_Weights
from tqdm import tqdm

In [47]:
csv_dir = '/Users/iyongjeong/WORK/dacon/clf/data/train_csv/train_file_list_100.csv'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = ImageDataset(csv_dir=csv_dir, transform=transform)
dataset.load_data()

dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

for images, labels in dataloader:
    print(images.shape, labels.shape)
    break

torch.Size([4, 3, 224, 224]) torch.Size([4])


In [48]:
# fine tunning
model = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)

In [53]:
def train(model, dataloader, epochs=10, learning_rate=0.001):
    import torch
    import torch.optim as optim
    import torch.nn as nn

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            
            # print shapes for debugging
            #print(f"Batch images shape: {images.shape}, labels shape: {labels.shape}")

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(dataloader):.4f}")

In [54]:
# train debug
train(model, dataloader, epochs=100, learning_rate=0.001)

Epoch [1/100], Loss: 0.3352
Epoch [2/100], Loss: 0.2524
Epoch [3/100], Loss: 0.0427
Epoch [4/100], Loss: 0.0576
Epoch [5/100], Loss: 0.1135
Epoch [6/100], Loss: 0.1005
Epoch [7/100], Loss: 0.0131
Epoch [8/100], Loss: 0.0067
Epoch [9/100], Loss: 0.0023
Epoch [10/100], Loss: 0.0013
Epoch [11/100], Loss: 0.0020
Epoch [12/100], Loss: 0.0008
Epoch [13/100], Loss: 0.0063
Epoch [14/100], Loss: 0.0048
Epoch [15/100], Loss: 0.0171
Epoch [16/100], Loss: 0.0257
Epoch [17/100], Loss: 0.0859
Epoch [18/100], Loss: 0.0211
Epoch [19/100], Loss: 0.0133
Epoch [20/100], Loss: 0.0678
Epoch [21/100], Loss: 0.2601
Epoch [22/100], Loss: 0.1573
Epoch [23/100], Loss: 0.1335
Epoch [24/100], Loss: 0.0617
Epoch [25/100], Loss: 0.0150
Epoch [26/100], Loss: 0.0065
Epoch [27/100], Loss: 0.0016
Epoch [28/100], Loss: 0.0011
Epoch [29/100], Loss: 0.0007
Epoch [30/100], Loss: 0.0143
Epoch [31/100], Loss: 0.0033
Epoch [32/100], Loss: 0.0069
Epoch [33/100], Loss: 0.0033
Epoch [34/100], Loss: 0.0009
Epoch [35/100], Loss: 0